# Document Similarity

# Rekomendasi Film dengan Kemiripan Dokumen

Sistem rekomendasi adalah salah satu aplikasi pembelajaran mesin yang populer dan paling banyak diadopsi. Biasanya digunakan untuk merekomendasikan entitas kepada pengguna dan entitas ini dapat berupa apa saja seperti produk, film, layanan, dan sebagainya.

Contoh rekomendasi yang populer meliputi,
- Amazon menyarankan produk di situs webnya
- Amazon Prime, Netflix, Hotstar merekomendasikan film\acara
- YouTube merekomendasikan video untuk ditonton

Biasanya sistem pemberi rekomendasi dapat diimplementasikan dalam tiga cara:

- Simple Rule-based Recommenders / Rekomendasi Berbasis Aturan Sederhana: Biasanya berdasarkan metrik dan ambang batas global tertentu seperti popularitas film, peringkat global, dll.
- Content-based Recommenders / Rekomendasi berbasis konten: Ini didasarkan pada penyediaan entitas serupa berdasarkan entitas minat tertentu. Metadata konten dapat digunakan di sini seperti deskripsi film, genre, pemeran, sutradara, dan sebagainya
- Collaborative filtering Recommenders / Pemfilteran kolaboratif Rekomendasi: Di ​​sini tidak diperlukan metadata, tetapi memprediksi rekomendasi dan peringkat berdasarkan peringkat sebelumnya dari berbagai pengguna dan item tertentu.

STKI / NLP paling relevan dengan Collaborative filtering Recommenders

## install library

In [1]:
# !pip install textsearch
# !pip install contractions
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

In [2]:
import pathlib
pathlib.Path().resolve()

WindowsPath('C:/Users/ASUS/JupyterNotebookFile/GHclone/Sistem-Temu-Kembali-Informasi/Minggu4 Vector Space/Projek MInggu 4')

## load dan view data

In [3]:
import pandas as pd

df = pd.read_csv("Highest Holywood Grossing Movies.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  1000 non-null   int64 
 1   Title                       1000 non-null   object
 2   Movie Info                  1000 non-null   object
 3   Year                        1000 non-null   int64 
 4   Distributor                 999 non-null    object
 5   Budget (in $)               1000 non-null   object
 6   Domestic Opening (in $)     1000 non-null   object
 7   Domestic Sales (in $)       1000 non-null   int64 
 8   International Sales (in $)  1000 non-null   int64 
 9   World Wide Sales (in $)     1000 non-null   int64 
 10  Release Date                1000 non-null   object
 11  Genre                       1000 non-null   object
 12  Running Time                1000 non-null   object
 13  License                     943 non-null    objec

In [4]:
df.head()

,Unnamed: 0,Title,Movie Info,Year,Distributor,Budget (in $),Domestic Opening (in $),Domestic Sales (in $),International Sales (in $),World Wide Sales (in $),Release Date,Genre,Running Time,License
0,0,Avatar,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000,77025481,785221649,2138484377,2923706026,16-Dec-09,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13
1,1,Avengers: Endgame,After the devastating events of Avengers: Infi...,2019,Walt Disney Studios Motion Pictures,356000000,357115007,858373000,1941066100,2799439100,24-Apr-19,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
2,2,Avatar: The Way of Water,Jake Sully lives with his newfound family form...,2022,20th Century Studios,December 14 2022 (EMEA APAC),134100226,684075767,1636174514,2320250281,24-Apr-19,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
3,3,Titanic,A seventeen-year-old aristocrat falls in love ...,1997,Paramount Pictures,200000000,28638131,674292608,1590450697,2264743305,19-Dec-97,"['Drama', 'Romance']",3 hr 14 min,PG-13
4,4,Star Wars: Episode VII - The Force Awakens,"As a new threat to the galaxy rises, Rey, a de...",2015,Walt Disney Studios Motion Pictures,245000000,247966675,936662225,1134647993,2071310218,16-Dec-15,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13


In [5]:
df.dropna(axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 942 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  942 non-null    int64 
 1   Title                       942 non-null    object
 2   Movie Info                  942 non-null    object
 3   Year                        942 non-null    int64 
 4   Distributor                 942 non-null    object
 5   Budget (in $)               942 non-null    object
 6   Domestic Opening (in $)     942 non-null    object
 7   Domestic Sales (in $)       942 non-null    int64 
 8   International Sales (in $)  942 non-null    int64 
 9   World Wide Sales (in $)     942 non-null    int64 
 10  Release Date                942 non-null    object
 11  Genre                       942 non-null    object
 12  Running Time                942 non-null    object
 13  License                     942 non-null    object

In [6]:
df_new = df.copy()
df_new = df_new[['Title', 'Year','Movie Info']] #- Genre
# df_new['description'] = df_new['Movie Info'].map(str) + ' ' + 'Genre are: '+df_new['Genre']
# df_new.dropna(inplace=True)
# # df = df.sort_values(by=['popularity'], ascending=False)
# df_new.info()

In [7]:
df_new.head()

,Title,Year,Movie Info
0,Avatar,2009,A paraplegic Marine dispatched to the moon Pan...
1,Avengers: Endgame,2019,After the devastating events of Avengers: Infi...
2,Avatar: The Way of Water,2022,Jake Sully lives with his newfound family form...
3,Titanic,1997,A seventeen-year-old aristocrat falls in love ...
4,Star Wars: Episode VII - The Force Awakens,2015,"As a new threat to the galaxy rises, Rey, a de..."


In [10]:
df_new.rename(columns={"Movie Info":"description"}, inplace=True)

In [11]:
df_new

,Title,Year,description
0,Avatar,2009,A paraplegic Marine dispatched to the moon Pan...
1,Avengers: Endgame,2019,After the devastating events of Avengers: Infi...
2,Avatar: The Way of Water,2022,Jake Sully lives with his newfound family form...
3,Titanic,1997,A seventeen-year-old aristocrat falls in love ...
4,Star Wars: Episode VII - The Force Awakens,2015,"As a new threat to the galaxy rises, Rey, a de..."
...,...,...,...
995,Sweet Home Alabama,2002,A young woman who has reinvented herself as a ...
996,Daddy's Home 2,2017,Having finally gotten used to each other's exi...
997,Hacksaw Ridge,2016,World War II American Army Medic Desmond T. Do...
998,Deja Vu,2006,"After a ferry is bombed in New Orleans, an A.T..."


In [12]:
df_new.iloc[200].description

'A greedy film producer assembles a team of moviemakers and sets out for the infamous Skull Island, where they find more than just cannibalistic natives.'

# Bangun Sistem Rekomendasi Film

Tahapan
- Pre Processing
- Feature Engineering
- Komputasi Doc Similarity
- Proses Retrieve
- proses rekomendasi film


## Kemiripan Dokumen / document similarity

Ada berbagai cara untuk menghitung kesamaan antara dua item dokumen. Salah satu ukuran yang paling banyak digunakan adalah __cosine similarity__ .

### Cosine Similarity

Cosine Similarity digunakan untuk menghitung skor numerik untuk menunjukkan kesamaan antara dua dokumen teks. Secara matematis, ini didefinisikan sebagai berikut:

$$ cosinus(x,y) = \frac{x. y^\intercal}{||x||.||y||} $$

In [13]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df_new['description']))
len(norm_corpus)

942

## Extrak TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(942, 2740)

## Compute Pairwise Document Similarity

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,932,933,934,935,936,937,938,939,940,941
0,1.000000,0.0,0.216635,0.0,0.000000,0.000000,0.00000,0.00000,0.0,0.000000,...,0.038791,0.000000,0.0,0.0,0.000000,0.036070,0.000000,0.051412,0.062036,0.0
1,0.000000,1.0,0.000000,0.0,0.000000,0.394828,0.02186,0.00000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.027225,0.000000,0.0
2,0.216635,0.0,1.000000,0.0,0.046471,0.015747,0.00000,0.00000,0.0,0.051040,...,0.000000,0.012022,0.0,0.0,0.038190,0.042753,0.017341,0.035461,0.000000,0.0
3,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.00000,0.00000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.000000,0.0,0.046471,0.0,1.000000,0.020390,0.00000,0.01596,0.0,0.017841,...,0.000000,0.015566,0.0,0.0,0.240443,0.040398,0.022455,0.000000,0.051223,0.0


## mendapatkan title / judul movies

In [16]:
movies_list = df['Title'].values
movies_list, movies_list.shape

(array(['Avatar', 'Avengers: Endgame', 'Avatar: The Way of Water',
        'Titanic', 'Star Wars: Episode VII - The Force Awakens',
        'Avengers: Infinity War', 'Spider-Man: No Way Home',
        'Jurassic World', 'The Lion King', 'The Avengers', 'Furious 7',
        'Top Gun: Maverick', 'Frozen II', 'Barbie',
        'Avengers: Age of Ultron', 'The Super Mario Bros. Movie',
        'Black Panther', 'Harry Potter and the Deathly Hallows: Part 2',
        'Star Wars: Episode VIII - The Last Jedi',
        'Jurassic World: Fallen Kingdom', 'Frozen', 'Beauty and the Beast',
        'Incredibles 2', 'The Fate of the Furious', 'Iron Man 3',
        'Minions', 'Captain America: Civil War', 'Aquaman',
        'The Lord of the Rings: The Return of the King', 'Skyfall',
        'Spider-Man: Far from Home', 'Captain Marvel',
        'Transformers: Dark of the Moon', 'Jurassic Park',
        'Transformers: Age of Extinction', 'The Dark Knight Rises',
        'Star Wars: Episode IX - The Rise

## Temukan Film Serupa Teratas untuk Contoh Film

Mari ambil __Minions__ film paling populer dari kerangka data di atas dan coba temukan film paling mirip yang dapat direkomendasikan

### ambil movie ID

In [17]:
movie_idx = np.where(movies_list == 'Avatar')[0][0]
movie_idx

0

### ambil similarities

In [18]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([1.        , 0.        , 0.21663452, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03288377, 0.        , 0.03460356, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.02814405, 0.        , 0.06389041, 0.02462479, 0.13096412,
       0.02095977, 0.        , 0.        , 0.02139398, 0.        ,
       0.08111821, 0.04219829, 0.07197649, 0.06699368, 0.        ,
       0.        , 0.        , 0.        , 0.01921193, 0.03903629,
       0.        , 0.0392738 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01793481, 0.02568271,
       0.02970357, 0.        , 0.        , 0.        , 0.02060315,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.043991  ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.02264

### Get top 5 similar movie IDs

In [19]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([  2, 328, 596, 534,  24], dtype=int64)

### Get top 5 similar movies

In [20]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['Avatar: The Way of Water', 'Prometheus', 'Borat', 'G-Force',
       'Iron Man 3'], dtype=object)

### Buat fungsi rekomendasi film untuk merekomendasikan 5 film serupa teratas untuk film apa pun


The movie title, movie title list and document similarity matrix dataframe akan diberikan sebagai input ke fungsi

In [21]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

# misal cari rekomendasi untuk film film berikut

In [36]:
popular_movies = ['Barbie','Frozen II', "Minions","Pirates of the Caribbean: The Curse of the Black Pearl","Fast & Furious 6"]

In [37]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()

Movie: Barbie
Top 5 recommended Movies: ['Batman v Superman: Dawn of Justice' 'Black Adam' 'The Incredibles'
 'Spirited Away' 'Arrival']

Movie: Frozen II
Top 5 recommended Movies: ['Journey to the Center of the Earth' 'Ghostbusters: Afterlife'
 'White House Down' 'The Super Mario Bros. Movie'
 'A Series of Unfortunate Events']

Movie: Minions
Top 5 recommended Movies: ['Divergent' 'Arrival' 'London Has Fallen' 'Resident Evil: Afterlife'
 'The Terminal']

Movie: Pirates of the Caribbean: The Curse of the Black Pearl
Top 5 recommended Movies: ['Pirates of the Caribbean: Dead Men Tell No Tales'
 'Pirates of the Caribbean: On Stranger Tides'
 "Pirates of the Caribbean: At World's End" 'White House Down' 'Taken']

Movie: Fast & Furious 6
Top 5 recommended Movies: ['Fast & Furious' 'X-Men Origins: Wolverine'
 'Mission: Impossible - Rogue Nation' 'RED' 'Deadpool 2']



# Rekomendasi Film (other method)

In [38]:
# Gensim FastText
from gensim.models import FastText

tokenized_docs = [doc.split() for doc in norm_corpus]
ft_model = FastText(tokenized_docs, window=30, min_count=2, workers=4, sg=1)

# Hasilkan penyematan tingkat dokumen / Word Embedding

Model penyematan kata (word embedding) memberi kita penyematan untuk setiap kata, bagaimana kita dapat menggunakannya untuk tugas ML\DL hilirisasi? salah satu caranya adalah dengan meratakannya atau menggunakan model sekuensial. Pendekatan yang lebih sederhana adalah dengan rata-rata semua penyematan kata untuk kata-kata dalam dokumen dan menghasilkan penyematan tingkat dokumen dengan panjang tetap (fixed-length document level emebdding)

In [39]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [40]:
doc_vecs_ft = averaged_word2vec_vectorizer(tokenized_docs, ft_model, 100)
doc_vecs_ft.shape

(942, 100)

# Dapatkan Rekomendasi Film

Memanfaatkan Cos Similarity untuk menghasilkan rekomendasi

In [41]:
doc_sim = cosine_similarity(doc_vecs_ft)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,932,933,934,935,936,937,938,939,940,941
0,1.000000,0.999994,0.999994,0.999986,0.999990,0.999995,0.999992,0.999990,0.999991,0.999991,...,0.999992,0.999990,0.999987,0.999985,0.999991,0.999982,0.999991,0.999991,0.999992,0.999984
1,0.999994,1.000000,0.999990,0.999986,0.999988,0.999995,0.999993,0.999987,0.999991,0.999988,...,0.999990,0.999988,0.999988,0.999983,0.999991,0.999981,0.999991,0.999992,0.999989,0.999980
2,0.999994,0.999990,1.000000,0.999987,0.999994,0.999995,0.999991,0.999990,0.999988,0.999990,...,0.999991,0.999988,0.999986,0.999987,0.999991,0.999986,0.999990,0.999990,0.999991,0.999985
3,0.999986,0.999986,0.999987,1.000000,0.999983,0.999988,0.999989,0.999984,0.999990,0.999979,...,0.999987,0.999988,0.999982,0.999982,0.999985,0.999984,0.999983,0.999985,0.999988,0.999983
4,0.999990,0.999988,0.999994,0.999983,1.000000,0.999993,0.999989,0.999990,0.999987,0.999989,...,0.999990,0.999986,0.999982,0.999984,0.999991,0.999988,0.999988,0.999988,0.999991,0.999978


In [42]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()

Movie: Barbie
Top 5 recommended Movies: ['Batman v Superman: Dawn of Justice' 'Black Adam' 'The Incredibles'
 'Arrival' 'Spirited Away']

Movie: Frozen II
Top 5 recommended Movies: ['Journey to the Center of the Earth' 'Ghostbusters: Afterlife'
 'White House Down' 'The Super Mario Bros. Movie'
 'Transformers: Revenge of the Fallen']

Movie: Minions
Top 5 recommended Movies: ['Divergent' 'Arrival' 'London Has Fallen' 'Resident Evil: Afterlife'
 'The Terminal']

Movie: Pirates of the Caribbean: The Curse of the Black Pearl
Top 5 recommended Movies: ['Pirates of the Caribbean: Dead Men Tell No Tales'
 'Pirates of the Caribbean: On Stranger Tides'
 "Pirates of the Caribbean: At World's End" 'White House Down'
 'Disclosure']

Movie: Fast & Furious 6
Top 5 recommended Movies: ['Fast & Furious' 'X-Men Origins: Wolverine'
 'Mission: Impossible - Rogue Nation' 'RED' 'Deadpool 2']

